# HDMR/Sobol Expansion Terms

Given a function $f : [0, 1]^d \rightarrow \mathbb{R}$ where $d = 3$

$$
f(\vec{Z}) = Z_1 + Z_2^2 + Z_1 Z_2 + Z_2 Z_3^2,
$$

and $Z_i \sim \mathcal{U}[0, 1]\ \forall i \in \{1, 2, 3 \}$, show
the Sobol representation of this function. 

The general Sobol representation is given by 

$$
\begin{aligned}
f_0 &= \int_\Gamma f(z)\ dz, \\
f_i(z_i) &= \int_{\Gamma^{d-1}} f(z)\ dz_{\sim i} - f_0, \\
f_{ij}(z_i, z_j) &= \int_{\Gamma^{d-2}} f(z)\ dz_{\sim ij} - f_i(z_i) - f_j(z_j) - f_0.
\end{aligned}
$$

on the domain $\Gamma$, in this case $\Gamma = [0, 1]^3$ and the notation $\{\sim i\}$
denotes all indices **except** $i$. 

Then the Sobol representation for our problem is 

$$
\begin{aligned}
f_0 &= \int_0^1 \int_0^1 \int_0^1 f(z)\ dz, && \text{(1)} \\
f_1(z_1) &= \int_0^1 \int_0^1 f(z)\ dz_2\ dz_3 - f_0, && \text{(2)} \\
f_2(z_2) &= \int_0^1 \int_0^1 f(z)\ dz_1\ dz_3 - f_0, && \text{(3)} \\
f_3(z_3) &= \int_0^1 \int_0^1 f(z)\ dz_1\ dz_2 - f_0, && \text{(4)} \\
f_{12}(z_1, z_2) &= \int_0^1 f(z)\ dz_3 - f_1(z_1) - f_2(z_2) - f_0, && \text{(5)}\\
f_{13}(z_1, z_3) &= \int_0^1 f(z)\ dz_2 - f_1(z_1) - f_3(z_3) - f_0, && \text{(6)} \\
f_{23}(z_2, z_3) &= \int_0^1 f(z)\ dz_1 - f_2(z_2) - f_3(z_3) - f_0. && \text{(7)}
\end{aligned}
$$

Here's some tedious solutions to the integrals:

$$
\begin{aligned}
f_0 &= \int_{Z_3 = 0}^1 \int_{Z_2 = 0}^1 \int_{Z_1 = 0}^1 z_1 + z_2^2 + z_1 z_2 + z_2 z_3^2\ dz_1\ dz_2\ dz_3 \\
    &= \int_{Z_3 = 0}^1 \int_{Z_2 = 0}^1 \left[ \frac{z_1^2}{2} + z_1 z_2^2 + \frac{z_1^2}{2} z_2 + z_1 z_2 z_3^2\right]_0^1 \ dz_2\ dz_3 \\
    &= \int_{Z_3 = 0}^1 \int_{Z_2 = 0}^1 \frac{1}{2} + z_2^2 + \frac{1}{2} z_2 + z_2 z_3^2 \ dz_2\ dz_3 \\
     &= \int_{Z_3 = 0}^1 \left[ \frac{z_2}{2} + \frac{z_2^3}{3} + \frac{z_2^2}{4} + \frac{z_2^2}{2} z_3^2\right]_0^1 \ dz_3 \\
     &= \int_{Z_3 = 0}^1 \frac{1}{2} + \frac{1}{3} + \frac{1}{4} + \frac{1}{2} z_3^2 \ dz_3 \\
     &= \left[ \frac{z_3}{2} + \frac{z_3}{3} + \frac{z_3}{4} + \frac{z_3^3}{6} \right]_0^1 \\
     &=  \frac{1}{2} + \frac{1}{3} + \frac{1}{4} + \frac{1}{6} \\
     &= \frac{5}{4}
\end{aligned}
$$

we all can integrate, so I just use mathematica to compute the remaining integrals (since i'm lazy).

In [32]:
from typing import Callable, Tuple

from numpy import ndarray
import numpy as np 

In [54]:
# Various function definitions from solving integrals
def f(Z):
    z1, z2, z3 = Z 
    return z1 + z2**2 +z1*z2 + z2*z3**2

def f0(Z):
    return 5/4 

def f1(Z):
    z1, z2, z3 = Z 
    return -3/4 + (3/2)*z1

def f2(Z):
    z1, z2, z3 = Z 
    return -5/4 + z2/3 + z2**2 + (1 + z2)/2 

def f3(Z):
    z1, z2, z3 = Z 
    return -5/12 + (1/2)*((1/2) + z3**2)

def f12(Z):
    z1, z2, z3 = Z 
    return 3/4 - z1/2 + (1/2)*(-1 - z2) + z1*z2 

def f13(Z):
    z1, z2, z3 = Z 
    return 1/4 - z1/2 + (1/2)*(-(1/2) - z3**2) + (1/2)*(z1 + z3**2)

def f23(Z):
    z1, z2, z3 = Z 
    return 5/12 + (1/2)*(-1 - z2) - z2/3 + (1 + z2)/2 + z2*z3**2 \
        + (1/2)*(-(1/2) - z3**2)

def monte_carlo_sampling(
        f: Callable, size: Tuple[int, int] = (3, ), 
        repeat: int = 100, n_samples: int = 1000) -> ndarray:
    f_repeat = []
    for _ in range(repeat):
        f_samples = []
        for _ in range(n_samples):
            Z = np.random.uniform(size=size)
            f_sample = f(Z)
            f_samples.append(f_sample)
        f_repeat.append(f_samples)
    return np.array(f_repeat)

def get_mc_var_mean_std(mc: ndarray) -> Tuple[float, float]:
    var = np.var(mc, axis=1)
    mean = np.mean(var)
    std = np.std(var)
    return mean, std 

In [46]:
# Monte carlo the various functions
f_repeat = monte_carlo_sampling(f) # D
f1_repeat = monte_carlo_sampling(f1) # D_i ...
f2_repeat = monte_carlo_sampling(f2)
f3_repeat = monte_carlo_sampling(f3)
f12_repeat = monte_carlo_sampling(f12) # D_ij ...
f13_repeat = monte_carlo_sampling(f13)
f23_repeat = monte_carlo_sampling(f23)

In [59]:
# You can check the mathematica notebook to verify that these agree
# with the exact integral
D_mean, D_std = get_mc_var_mean_std(f_repeat)
D1_mean, D1_std = get_mc_var_mean_std(f1_repeat)
D2_mean, D2_std = get_mc_var_mean_std(f2_repeat)
D3_mean, D3_std = get_mc_var_mean_std(f3_repeat)
D12_mean, D12_std = get_mc_var_mean_std(f12_repeat)
D13_mean, D13_std = get_mc_var_mean_std(f13_repeat)
D23_mean, D23_std = get_mc_var_mean_std(f23_repeat)

print_mean_std = lambda mean, std: print("%.4f +/- %.4f" % (mean, std))

print_mean_std(D_mean, D_std)

print_mean_std(D1_mean, D1_std)
print_mean_std(D2_mean, D2_std)
print_mean_std(D3_mean, D3_std)

print_mean_std(D12_mean, D12_std)
print_mean_std(D13_mean, D13_std)
print_mean_std(D23_mean, D23_std)

0.5115 +/- 0.0196
0.1864 +/- 0.0051
0.2852 +/- 0.0084
0.0222 +/- 0.0008
0.0069 +/- 0.0003
0.0000 +/- 0.0000
0.0074 +/- 0.0004


In [61]:
# Compute sobol indices
sobol = lambda D_ids, D: D_ids/D 
print(sobol(D1_mean, D_mean))
print(sobol(D2_mean, D_mean))
print(sobol(D3_mean, D_mean))
print(sobol(D12_mean, D_mean))
print(sobol(D13_mean, D_mean))
print(sobol(D23_mean, D_mean))

0.3644134438325231
0.5576185146535788
0.043455224120648325
0.013536872531943692
2.3441422159366305e-33
0.014456158042735714
